<img src="idal-logo.png" align="right" style="float" width="400">
<font color="#CA3532"><h1 align="left">mIA3</h1></font>
<font color="#6E6E6E"><h2 align="left">Tarea Evaluable. Aprendizaje Profundo 1 y 2 (Parte 2 de 3).</h2></font> 

#### Elaborado por Felipe Ramírez Herrera

In [1]:
import torch
import torch.nn as nn
import os

import dl_common as mc
import dl_gcnn_models as mg
import dl_tfmr_models as mt
import dl_xxxx_models as mm
import dl_greedy_search as mgs

import pandas as pd

Computing device = cuda


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:

torch.manual_seed(1234)

In [4]:
#DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    !nvidia-smi

Mon Apr  8 10:54:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.43.02              Driver Version: 535.98       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060        On  | 00000000:08:00.0  On |                  N/A |
|  0%   49C    P8              18W / 170W |   1107MiB / 12288MiB |     12%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
import torchtext
import torchtext.data
import torchtext.data.datasets_utils
import torchtext.datasets


if os.path.exists(mc.en_vocab_file):
    mc.en_vocabulary = torch.load(mc.en_vocab_file)
else:
    raise Exception("")

if os.path.exists(mc.es_vocab_file):
    mc.es_vocabulary = torch.load(mc.es_vocab_file)
else:
    raise Exception("")

if os.path.exists(mc.fr_vocab_file):
    mc.fr_vocabulary = torch.load(mc.fr_vocab_file)
else:
    raise Exception("")

In [7]:
print("EN Vocabulary Size = {0}".format(len(mc.en_vocabulary)))
print("ES Vocabulary Size = {0}".format(len(mc.es_vocabulary)))
print("FR Vocabulary Size = {0}".format(len(mc.fr_vocabulary)))

EN Vocabulary Size = 74563
ES Vocabulary Size = 91234
FR Vocabulary Size = 86462


In [8]:
mc.PAD_EN_IDX = mc.en_vocabulary[mc.PAD_WORD]
mc.BOS_EN_IDX = mc.en_vocabulary[mc.BOS_WORD]
mc.EOS_EN_IDX = mc.en_vocabulary[mc.EOS_WORD]
mc.UNK_EN_IDX = mc.en_vocabulary[mc.UNK_WORD]


mc.PAD_ES_IDX = mc.es_vocabulary[mc.PAD_WORD]
mc.BOS_ES_IDX = mc.es_vocabulary[mc.BOS_WORD]
mc.EOS_ES_IDX = mc.es_vocabulary[mc.EOS_WORD]
mc.UNK_ES_IDX = mc.es_vocabulary[mc.UNK_WORD]

mc.PAD_FR_IDX = mc.fr_vocabulary[mc.PAD_WORD]
mc.BOS_FR_IDX = mc.fr_vocabulary[mc.BOS_WORD]
mc.EOS_FR_IDX = mc.fr_vocabulary[mc.EOS_WORD]
mc.UNK_FR_IDX = mc.fr_vocabulary[mc.UNK_WORD]

mc.en_vocab_size = len(mc.en_vocabulary)
mc.es_vocab_size = len(mc.es_vocabulary)
mc.fr_vocab_size = len(mc.fr_vocabulary)

In [9]:
import random
import pickle
from collections import Counter 


max_len = 0
min_len = 4500

fe = os.path.exists(mc.full_data_file)
te = os.path.exists(mc.trn_data_file)
ve = os.path.exists(mc.val_data_file)
tt = os.path.exists(mc.tst_data_file)

if fe and te and ve and tt:
    with open(mc.full_data_file, 'rb') as fp:
            mc.full_data = pickle.load(fp)
    with open(mc.trn_data_file, 'rb') as fp:
            mc.trn_subset = pickle.load(fp)
    with open(mc.val_data_file, 'rb') as fp:
            mc.val_subset = pickle.load(fp)        
    with open(mc.tst_data_file, 'rb') as fp:
            mc.tst_subset = pickle.load(fp)           
    for (en_seq,es_seq,fr_seq) in mc.full_data:     
        a = len(en_seq)
        b = len(es_seq)
        c = len(fr_seq) 
        max_len = max(max_len, a, b, c)    
        min_len = min(min_len, a, b, c)    
else:
    raise Exception("")

if (mc.max_seq_length < max_len):
    mc.max_seq_length = max_len + 2

if (mc.min_seq_length > min_len):
    mc.min_seq_length = min_len + 2


print("Selected Records: {0}".format(len(mc.full_data)))
print("MAX SEQ LEN {0}".format(mc.max_seq_length))
print("MIN SEQ LEN {0}".format(mc.min_seq_length))

size_of_trn_set_size = len(mc.trn_subset)
size_of_val_set_size = len(mc.val_subset)
size_of_tst_set_size = len(mc.tst_subset)

print("Training tuples: {0} Validation tuples: {1} Testing tuples: {2}".format(size_of_trn_set_size, size_of_val_set_size, size_of_tst_set_size))


Selected Records: 440944
MAX SEQ LEN 102
MIN SEQ LEN 7
Training tuples: 308660 Validation tuples: 88630 Testing tuples: 43654


In [10]:
from torch.utils.data import DataLoader

import dl_common_processing as mp
import math
Bilingual_EN_to_ES_tstset = DataLoader(mp.LanguageDataset(mc.tst_subset, size_of_tst_set_size), batch_size=mc.BATCH_SIZE, shuffle=False, collate_fn=mp.generate_batch_EN_ES)
Bilingual_EN_to_FR_tstset = DataLoader(mp.LanguageDataset(mc.tst_subset, size_of_tst_set_size), batch_size=mc.BATCH_SIZE, shuffle=False, collate_fn=mp.generate_batch_EN_FR)
Trilingual_tstset = DataLoader(mp.LanguageDataset(mc.tst_subset, size_of_tst_set_size), batch_size=mc.BATCH_SIZE, shuffle=False, collate_fn=mp.generate_batch)

batches_for_testing = math.ceil( size_of_tst_set_size / mc.BATCH_SIZE)


## Funcionalidad compartida

In [11]:
st_metrics = []
dt_metrics = []

#### Transformers

In [12]:

EMB_SIZE = 256
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

base_EN_ES_model_path = 'tfmr_single_en_es_{0}.pt'
base_EN_FR_model_path = 'tfmr_single_en_fr_{0}.pt'
dual_model_path = 'tfmr_dual_en_es_fr_{0}.pt'
dual_model_path_WS = 'tfmr_dual_en_es_fr_weight_sharing_{0}.pt'

transformer_base_EN_to_ES = mt.Seq2SeqTransformer("TFMR_EN_ES", NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, NHEAD, mc.en_vocab_size, mc.es_vocab_size, FFN_HID_DIM)
transformer_base_EN_to_FR = mt.Seq2SeqTransformer("TFMR_EN_FR",NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, NHEAD, mc.en_vocab_size, mc.fr_vocab_size, FFN_HID_DIM)
transformer_dual_EN_to_ES_and_FS = mt.DoubleTaskSeq2SeqTransformer("TFMR_DUAL_EN_ES_FR", NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, NHEAD, mc.en_vocab_size, mc.es_vocab_size, mc.fr_vocab_size, FFN_HID_DIM)
transformer_dual_EN_to_ES_and_FS_with_WS = mt.DoubleTaskSeq2SeqTransformer("TFMR_DUAL_EN_ES_FR (WS)", NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, NHEAD, mc.en_vocab_size, mc.es_vocab_size, mc.fr_vocab_size, FFN_HID_DIM, weight_sharing=True)





/home/framirez/miniconda3/envs/DL_Env/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [13]:
transformer_base_EN_to_ES.to(mm.DEVICE)

Seq2SeqTransformer(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-2): 3 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=512, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0-2): 3 x TransformerDecoderLayer(
          (self_attn): MultiheadAttent

In [14]:
transformer_base_EN_to_FR.to(mm.DEVICE)

Seq2SeqTransformer(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-2): 3 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=512, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0-2): 3 x TransformerDecoderLayer(
          (self_attn): MultiheadAttent

In [15]:
transformer_dual_EN_to_ES_and_FS.to(mm.DEVICE)

DoubleTaskSeq2SeqTransformer(
  (transformer): CustomDoubleDecoderTransformer(
    (encoder): CustomTransformerEncoder(
      (layers): ModuleList(
        (0-2): 3 x CustomTransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=512, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
          (activation): ReLU()
        )
      )
      (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
    (decoder_a): CustomTransformerDecoder(
      (layers): ModuleList(
  

In [16]:
transformer_dual_EN_to_ES_and_FS_with_WS.to(mm.DEVICE)

DoubleTaskSeq2SeqTransformer(
  (transformer): CustomDoubleDecoderTransformer(
    (encoder): CustomTransformerEncoder(
      (layers): ModuleList(
        (0-2): 3 x CustomTransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=512, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
          (activation): ReLU()
        )
      )
      (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
    (decoder_a): CustomTransformerDecoder(
      (layers): ModuleList(
  

In [17]:
def load_model_from(model, filename):
    checkpoint = torch.load(filename.format(mc.NUMBER_OF_EPOCHS - 1))
    if (checkpoint['epoch'] == mc.NUMBER_OF_EPOCHS - 1):
        model.load_state_dict(checkpoint['model_state_dict'])
    else:
        raise RuntimeError("Checkpoint {0} is corrupt".format(mc.NUMBER_OF_EPOCHS -1))

In [18]:
# Carga el último model cargado


load_model_from(transformer_base_EN_to_ES,  base_EN_ES_model_path)
load_model_from(transformer_base_EN_to_FR,  base_EN_FR_model_path)
load_model_from(transformer_dual_EN_to_ES_and_FS,  dual_model_path)
load_model_from(transformer_dual_EN_to_ES_and_FS_with_WS,  dual_model_path_WS)


In [19]:
# Inferencia
import warnings
warnings.filterwarnings('ignore')

models = [transformer_base_EN_to_ES, transformer_base_EN_to_FR]

src_langs = [mc.Language.EN, mc.Language.EN]
tgt_langs = [mc.Language.ES, mc.Language.FR]

hist = mgs.run_tfrmr_st_inference(mc.tst_subset, models, src_langs, tgt_langs)

st_metrics.append(pd.DataFrame.from_dict(hist, orient='index'))
        

In [20]:
hist = mgs.run_tfrmr_dt_inference(mc.tst_subset, [transformer_dual_EN_to_ES_and_FS, transformer_dual_EN_to_ES_and_FS_with_WS])

dt_metrics.append(pd.DataFrame.from_dict(hist, orient='index'))

#### Gated Convolutional Networks

In [21]:
INPUT_ES_VOCAB_DIM = mc.en_vocab_size
OUTPUT_ES_VOCAB_DIM = mc.es_vocab_size
OUTPUT_FR_VOCAB_DIM = mc.fr_vocab_size
GCNN_EMBDIM = 256
GCNN_HID_DIM = 512 # each conv. layer has 2 * hid_dim filters
GCNN_ENC_LAYERS = 5 # number of conv. blocks in encoder
GCNN_DEC_LAYERS = 5 # number of conv. blocks in decoder
GCNN_ENC_KERNEL_SIZE = 3 # must be odd!
GCNN_DEC_KERNEL_SIZE = 3 # can be even or odd
GCNN_ENC_DROPOUT = 0.25
GCNN_DEC_DROPOUT = 0.25

CNN_TRG_PAD_IDX = mc.PAD_ES_IDX

CNN_EN_ES_model_path = 'gcnn_mt_en_es_{0}.pt'
CNN_EN_FR_model_path = 'gcnn_mt__en_fr_{0}.pt'
CNN_EN_ES_FR_model_path = 'gcnn_mt_en_es_fr_{0}.pt'
CNN_EN_ES_FR_WS_model_path = 'gcnn_mt_ws_en_es_fr_{0}.pt'

CNN_ES_enc = mg.GatedConvEncoder(INPUT_ES_VOCAB_DIM, GCNN_EMBDIM, GCNN_HID_DIM, GCNN_ENC_LAYERS, GCNN_ENC_KERNEL_SIZE, GCNN_ENC_DROPOUT, mm.DEVICE, mc.max_seq_length)
CNN_ES_dec = mg.GatedConvDecoder(OUTPUT_ES_VOCAB_DIM, GCNN_EMBDIM, GCNN_HID_DIM, GCNN_DEC_LAYERS, GCNN_DEC_KERNEL_SIZE, GCNN_DEC_DROPOUT, CNN_TRG_PAD_IDX, mm.DEVICE, mc.max_seq_length)

CNN_FR_enc = mg.GatedConvEncoder(INPUT_ES_VOCAB_DIM, GCNN_EMBDIM, GCNN_HID_DIM, GCNN_ENC_LAYERS, GCNN_ENC_KERNEL_SIZE, GCNN_ENC_DROPOUT, mm.DEVICE, mc.max_seq_length)
CNN_FR_dec = mg.GatedConvDecoder(OUTPUT_FR_VOCAB_DIM, GCNN_EMBDIM, GCNN_HID_DIM, GCNN_DEC_LAYERS, GCNN_DEC_KERNEL_SIZE, GCNN_DEC_DROPOUT, CNN_TRG_PAD_IDX, mm.DEVICE, mc.max_seq_length)

CNN_model_EN_ES = mg.GatedConvSeq2Seq("CNN_EN_ES", CNN_ES_enc, CNN_ES_dec)
CNN_model_EN_FR = mg.GatedConvSeq2Seq("CNN_EN_FR", CNN_FR_enc, CNN_FR_dec)

CNN_DUAL_enc = mg.GatedConvEncoder(INPUT_ES_VOCAB_DIM, GCNN_EMBDIM, GCNN_HID_DIM, GCNN_ENC_LAYERS, GCNN_ENC_KERNEL_SIZE, GCNN_ENC_DROPOUT, mm.DEVICE, mc.max_seq_length)
CNN_DUAL_ES_dec = mg.GatedConvDecoder(OUTPUT_ES_VOCAB_DIM, GCNN_EMBDIM, GCNN_HID_DIM, GCNN_DEC_LAYERS, GCNN_DEC_KERNEL_SIZE, GCNN_DEC_DROPOUT, CNN_TRG_PAD_IDX, mm.DEVICE, mc.max_seq_length)
CNN_DUAL_FR_dec = mg.GatedConvDecoder(OUTPUT_FR_VOCAB_DIM, GCNN_EMBDIM, GCNN_HID_DIM, GCNN_DEC_LAYERS, GCNN_DEC_KERNEL_SIZE, GCNN_DEC_DROPOUT, CNN_TRG_PAD_IDX, mm.DEVICE, mc.max_seq_length)
CNN_DUAL_model = mg.GatedConvDualTaskSeq2Seq("CNN_EN_ES_FR", CNN_DUAL_enc, CNN_DUAL_ES_dec, CNN_DUAL_FR_dec)

CNN_DUAL_WS_enc = mg.GatedConvEncoder(INPUT_ES_VOCAB_DIM, GCNN_EMBDIM, GCNN_HID_DIM, GCNN_ENC_LAYERS, GCNN_ENC_KERNEL_SIZE, GCNN_ENC_DROPOUT, mm.DEVICE, mc.max_seq_length)
CNN_DUAL_WS_ES_dec = mg.GatedConvDecoder(OUTPUT_ES_VOCAB_DIM, GCNN_EMBDIM, GCNN_HID_DIM, GCNN_DEC_LAYERS, GCNN_DEC_KERNEL_SIZE, GCNN_DEC_DROPOUT, CNN_TRG_PAD_IDX, mm.DEVICE, mc.max_seq_length)
CNN_DUAL_WS_FR_dec = mg.GatedConvDecoder(OUTPUT_FR_VOCAB_DIM, GCNN_EMBDIM, GCNN_HID_DIM, GCNN_DEC_LAYERS, GCNN_DEC_KERNEL_SIZE, GCNN_DEC_DROPOUT, CNN_TRG_PAD_IDX, mm.DEVICE, mc.max_seq_length)
CNN_DUAL_WS_model = mg.GatedConvDualTaskSeq2Seq("CNN_EN_ES_FR (WS)", CNN_DUAL_WS_enc, CNN_DUAL_WS_ES_dec, CNN_DUAL_WS_FR_dec, shared_weights=True)



In [22]:
load_model_from(CNN_model_EN_ES,  CNN_EN_ES_model_path)
load_model_from(CNN_model_EN_FR,  CNN_EN_FR_model_path)
load_model_from(CNN_DUAL_model,  CNN_EN_ES_FR_model_path)
load_model_from(CNN_DUAL_WS_model,  CNN_EN_ES_FR_WS_model_path)

In [23]:
CNN_model_EN_ES.to(mm.DEVICE)

GatedConvSeq2Seq(
  (encoder): GatedConvEncoder(
    (tok_embedding): Embedding(74563, 256)
    (pos_embedding): Embedding(102, 256)
    (incoming_projection): Linear(in_features=256, out_features=512, bias=True)
    (outgoing_projection): Linear(in_features=512, out_features=256, bias=True)
    (convs): ModuleList(
      (0-4): 5 x Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
    )
    (dropout): Dropout(p=0.25, inplace=False)
  )
  (decoder): GatedConvDecoder(
    (tok_embedding): Embedding(91234, 256)
    (pos_embedding): Embedding(102, 256)
    (dec_incoming_projection): Linear(in_features=256, out_features=512, bias=True)
    (dec_outgoing_projection): Linear(in_features=512, out_features=256, bias=True)
    (attn_incoming_projection): Linear(in_features=512, out_features=256, bias=True)
    (attn_outgoing_projection): Linear(in_features=256, out_features=512, bias=True)
    (fc_out): Linear(in_features=256, out_features=91234, bias=True)
    (convs): ModuleList(

In [24]:
CNN_model_EN_FR.to(mm.DEVICE)

GatedConvSeq2Seq(
  (encoder): GatedConvEncoder(
    (tok_embedding): Embedding(74563, 256)
    (pos_embedding): Embedding(102, 256)
    (incoming_projection): Linear(in_features=256, out_features=512, bias=True)
    (outgoing_projection): Linear(in_features=512, out_features=256, bias=True)
    (convs): ModuleList(
      (0-4): 5 x Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
    )
    (dropout): Dropout(p=0.25, inplace=False)
  )
  (decoder): GatedConvDecoder(
    (tok_embedding): Embedding(86462, 256)
    (pos_embedding): Embedding(102, 256)
    (dec_incoming_projection): Linear(in_features=256, out_features=512, bias=True)
    (dec_outgoing_projection): Linear(in_features=512, out_features=256, bias=True)
    (attn_incoming_projection): Linear(in_features=512, out_features=256, bias=True)
    (attn_outgoing_projection): Linear(in_features=256, out_features=512, bias=True)
    (fc_out): Linear(in_features=256, out_features=86462, bias=True)
    (convs): ModuleList(

In [25]:
CNN_DUAL_model.to(mm.DEVICE)

GatedConvDualTaskSeq2Seq(
  (encoder): GatedConvEncoder(
    (tok_embedding): Embedding(74563, 256)
    (pos_embedding): Embedding(102, 256)
    (incoming_projection): Linear(in_features=256, out_features=512, bias=True)
    (outgoing_projection): Linear(in_features=512, out_features=256, bias=True)
    (convs): ModuleList(
      (0-4): 5 x Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
    )
    (dropout): Dropout(p=0.25, inplace=False)
  )
  (decoder_a): GatedConvDecoder(
    (tok_embedding): Embedding(91234, 256)
    (pos_embedding): Embedding(102, 256)
    (dec_incoming_projection): Linear(in_features=256, out_features=512, bias=True)
    (dec_outgoing_projection): Linear(in_features=512, out_features=256, bias=True)
    (attn_incoming_projection): Linear(in_features=512, out_features=256, bias=True)
    (attn_outgoing_projection): Linear(in_features=256, out_features=512, bias=True)
    (fc_out): Linear(in_features=256, out_features=91234, bias=True)
    (convs): M

In [26]:
CNN_DUAL_WS_model.to(mm.DEVICE)

GatedConvDualTaskSeq2Seq(
  (encoder): GatedConvEncoder(
    (tok_embedding): Embedding(74563, 256)
    (pos_embedding): Embedding(102, 256)
    (incoming_projection): Linear(in_features=256, out_features=512, bias=True)
    (outgoing_projection): Linear(in_features=512, out_features=256, bias=True)
    (convs): ModuleList(
      (0-4): 5 x Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
    )
    (dropout): Dropout(p=0.25, inplace=False)
  )
  (decoder_a): GatedConvDecoder(
    (tok_embedding): Embedding(91234, 256)
    (pos_embedding): Embedding(102, 256)
    (dec_incoming_projection): Linear(in_features=256, out_features=512, bias=True)
    (dec_outgoing_projection): Linear(in_features=512, out_features=256, bias=True)
    (attn_incoming_projection): Linear(in_features=512, out_features=256, bias=True)
    (attn_outgoing_projection): Linear(in_features=256, out_features=512, bias=True)
    (fc_out): Linear(in_features=256, out_features=91234, bias=True)
    (convs): M

In [27]:
# Inferencia
import warnings
warnings.filterwarnings('ignore')

models = [CNN_model_EN_ES, CNN_model_EN_FR]

src_langs = [mc.Language.EN, mc.Language.EN]
tgt_langs = [mc.Language.ES, mc.Language.FR]

hist = mgs.run_gccn_st_inference(mc.tst_subset, models, src_langs, tgt_langs)

st_metrics.append(pd.DataFrame.from_dict(hist, orient='index'))
        

Model: CNN_EN_ES
1.SRC_SEQ:   in its submissions of   february and   june   the state party contests any violations of the covenant
1.TGT_SEQ:   en sus exposiciones de   de febrero y   de junio de   el estado parte impugna que se haya violado el pacto
1.OUT_SEQ:  en sus observaciones de   de febrero y   de junio de   el estado parte se refiere a las violaciones de
2.SRC_SEQ:   targetgroup stations these are aimed at a specific target group within the flemish community
2.TGT_SEQ:   las emisoras destinadas a grupos concretos estan dirigidas a un grupo particular en la comunidad flamenca
2.OUT_SEQ:  las estaciones de alta comercio se trata de un grupo especifico de objetivo en la que se
3.SRC_SEQ: we consider it a promising sign that support has been coming from member states belonging to various groups
3.TGT_SEQ: consideramos que esa propuesta es un signo prometedor del apoyo que vienen prestando los estados miembros pertenecientes a diversos grupos
3.OUT_SEQ:  considerando que se ha sen

IndexError: Replacement index 3 out of range for positional args tuple

In [ ]:
hist = mgs.run_gccn_dt_inference(mc.tst_subset, [CNN_DUAL_model, CNN_DUAL_WS_model])

dt_metrics.append(pd.DataFrame.from_dict(hist, orient='index'))

In [ ]:
results_for_st = pd.concat(st_metrics)
results_for_st.to_csv("./st_inference.csv", sep=';', index=True, encoding='utf-8')

print(results_for_st)

results_for_dt = pd.concat(dt_metrics)
results_for_dt.to_csv("./dt_inference.csv", sep=';', index=True, encoding='utf-8')

print(results_for_dt)